# Matern Interpolation on a one dimensional example

In [1]:
using PyCall
using LaplaceInterpolation
using SparseArrays, LinearAlgebra
using Plots, Random

┌ Info: Precompiling LaplaceInterpolation [afb1b68e-e51f-4e49-a799-0e085170423d]
└ @ Base loading.jl:1278
ERROR: LoadError: LoadError: syntax: incomplete: unterminated multi-line comment #= ... =#
Stacktrace:
 [1] top-level scope at /Users/charlottehaley/.julia/packages/LaplaceInterpolation/178Ul/src/Matern1D2D.jl:319
 [2] include(::Function, ::Module, ::String) at ./Base.jl:380
 [3] include at ./Base.jl:368 [inlined]
 [4] include(::String) at /Users/charlottehaley/.julia/packages/LaplaceInterpolation/178Ul/src/LaplaceInterpolation.jl:1
 [5] top-level scope at /Users/charlottehaley/.julia/packages/LaplaceInterpolation/178Ul/src/LaplaceInterpolation.jl:5
 [6] include(::Function, ::Module, ::String) at ./Base.jl:380
 [7] include(::Module, ::String) at ./Base.jl:368
 [8] top-level scope at none:2
 [9] eval at ./boot.jl:347 [inlined]
 [10] eval(::Expr) at ./client.jl:467
 [11] top-level scope at ./none:3
in expression starting at /Users/charlottehaley/.julia/packages/LaplaceInterpolation/1

LoadError: [91mFailed to precompile LaplaceInterpolation [afb1b68e-e51f-4e49-a799-0e085170423d] to /Users/charlottehaley/.julia/compiled/v1.5/LaplaceInterpolation/0zvkA_CtSfZ.ji.[39m

This next cell assumes your python installation has [`scikit-learn`](https://scikit-learn.org/stable/install.html) available.  

In [2]:
py"""
import numpy as np
from matplotlib import pyplot as plt

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, Matern

np.random.seed(1)

def f(x):
    return x * np.sin(x)
    

def matern_data_1d():
    X = np.atleast_2d([1.,2., 3., 4.,5., 6., 7., 8.]).T
    y = f(X).ravel()
    kernel = 1.0 * Matern(length_scale=1.0, nu=1.5)
    gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9)
    gp.fit(X, y)
    x = np.atleast_2d(np.linspace(0, 10, 1000)).T
# Make the prediction on the meshed x-axis (ask for MSE as well)
    y_pred, sigma = gp.predict(x, return_std=True)
    return y_pred, sigma
"""
y_pred, sigma = py"matern_data_1d"()

LoadError: [91mPyError ($(Expr(:escape, :(ccall(#= /Users/charlottehaley/.julia/packages/PyCall/BD546/src/pyeval.jl:38 =# @pysym(:PyEval_EvalCode), PyPtr, (PyPtr, PyPtr, PyPtr), o, globals, locals))))) <class 'ModuleNotFoundError'>[39m
[91mModuleNotFoundError("No module named 'sklearn'")[39m
[91m  File "/Users/charlottehaley/.julia/packages/PyCall/BD546/src/pyeval.jl", line 4, in <module>[39m


In [4]:
plot(y_pred, ylim=[-10,20])

LoadError: [91mUndefVarError: y_pred not defined[39m

In [5]:
number_of_gridpoints = 1000
No_of_nodes_discarded = 900
random_permutation = randperm(MersenneTwister(1234), number_of_gridpoints)
discard = random_permutation[1:No_of_nodes_discarded]
# discard = rand(2:length(y_pred)-1, No_of_nodes_discarded);
# discard = unique(discard);
missing_data = copy(y_pred)
missing_data[discard].=-100
Plots.plot(missing_data, ylim=[-10,20])
len = length(y_pred)
h = 1e-2
nu = 1.5
length_scale = 1.0
dimension = 1
#=
C = sparse(I, len, len)
for i in discard
    C[i,i] = 0;
end
f = C*y_pred;
=#
#To do -- random permutation of 1000 and remove the first 90% of the data.
#Random permutation
#Set the seed for consistent result

LoadError: [91mUndefVarError: y_pred not defined[39m

In [6]:
m = dimension/2+nu
epsilon = sqrt(2*nu)/length_scale

restored_data_laplace, restored_data_matern = Matern_1D_Grid(y_pred, h, discard, 
    m, epsilon)
# f = y_pred[:];
# u =((C-(Id -C)*A1D)) \ (C*f);
# restored_data_laplace = u;
# A = ∇²1d_Grid(len, h);
# epsilon = sqrt(2*nu)/length_scale
# sizeA = size(A,1);
# for i = 1:sizeA
#     A[i,i] = A[i,i] + epsilon^2
# end

# A2DM = A*A;
# u =((C-(Id -C)*A2DM)) \ (C*f);
# restored_data_matern = u

LoadError: [91mUndefVarError: dimension not defined[39m

In [7]:

Plot_Matrix =[y_pred, restored_data_laplace, restored_data_matern]
#plot(y_pred, label="Original Data")
# #plot!(missing_data, title="Data with missing points");
# Plots.plot(restored_data_laplace, label="Laplace interpolated data");
# Plots.plot!(restored_data_matern, label="Matern, m = 2, eps = 0.1");
# #Plots.plot(plot1, plot2, plot3, plot4, layout = (2, 2), legend = false)
plot(discard, -5*ones(size(discard)), seriestype = :scatter, label = "Missing Data")
plot!(Plot_Matrix, labels=["Original Data" "Laplace Interpolation" "Matern Interpolation"], legend=:topleft, lw = 3)
#png("1D_MaternInterpolation.png")

LoadError: [91mUndefVarError: y_pred not defined[39m

In [8]:
error_laplace = abs.(restored_data_laplace - y_pred)./y_pred;
error_matern = abs.(restored_data_matern - y_pred)./y_pred;
Errors = [error_laplace, error_matern];

LoadError: [91mUndefVarError: restored_data_laplace not defined[39m

In [9]:
plot(Errors, labels=["Laplace Interpolation Error" "Matern Interpolation Error"], legend=:topleft)
#png("1D_InterpolationError.png")

LoadError: [91mUndefVarError: Errors not defined[39m

In [10]:
print(error_matern)

LoadError: [91mUndefVarError: error_matern not defined[39m